In [ ]:
# The goal is to download the html of a page that shows the standigs of the epl.
#The requests library will help in making HTTP requests to the website with our data.
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup


#settings list of years we want to scrap.
years = list(range(2024, 2011, -1))

# This list holds multiple dataframes, each dataframe will contain match logs of one team in one season.
all_matches = []

#url
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


# Loop through the years
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select("table.stats_table")[0]
    links = [l.get("href") for l in standings_table.find_all("a")]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"http://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", " ").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [i.get("href") for i in soup.find_all("a")]
        links = [l for l in links if l and "all_comps/shooting/" in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        # Insert the code to scrape passing data here
        passing_soup = BeautifulSoup(data.text)
        passing_a_tags = passing_soup.find_all('a')
        passings_hrefs = [l.get("href") for l in passing_a_tags]
        passing_links = [l for l in passings_hrefs if l and 'all_comps/passing/' in l]

        if not passing_links:
            continue

        passing_url = f"https://fbref.com{passing_links[0]}"  # Use the first passing link
        data = requests.get(passing_url)
        passing = pd.read_html(data.text, match="Passing")[0]
        passing.columns = passing.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "FK", "Dist", "PK", "PKatt"]], on="Date").merge(
                passing[["Date", "Time", "Cmp", "Att", "Cmp%", "TotDist", "PrgDist", "Cmp"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(7)

In [2]:
#Merging the data using dataframe
import pandas as pd

# Assuming all_matches is a list of DataFrames
data = pd.concat(all_matches, ignore_index=True)

In [3]:
data

,Date,Time_x,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Cmp%,Cmp%,TotDist,PrgDist,Cmp,Cmp,Cmp,Cmp,Season,Team
0,2023-08-13,14:00,Premier League,Matchweek 1,Sun,Away,D,2,2,Brentford,...,88.4,40.5,9253.0,2521.0,658.0,404.0,214.0,17.0,2024,Tottenham Hotspur
1,2023-08-19,17:30,Premier League,Matchweek 2,Sat,Home,W,2,0,Manchester Utd,...,86.4,66.1,7812.0,2826.0,460.0,198.0,203.0,37.0,2024,Tottenham Hotspur
2,2023-08-26,12:30,Premier League,Matchweek 3,Sat,Away,W,2,0,Bournemouth,...,90.4,59.2,8012.0,3260.0,498.0,262.0,198.0,29.0,2024,Tottenham Hotspur
3,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Away,W,5,2,Burnley,...,90.6,56.9,7886.0,2647.0,505.0,272.0,183.0,37.0,2024,Tottenham Hotspur
4,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Home,W,2,1,Sheffield Utd,...,87.8,58.3,9648.0,3295.0,579.0,291.0,245.0,35.0,2024,Tottenham Hotspur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,2018-04-15,16:00,Premier League,Matchweek 34,Sun,Away,W,1,0,Manchester Utd,...,83.8,44.7,4659.0,2088.0,236.0,104.0,93.0,34.0,2018,West Bromwich Albion
4716,2018-04-21,12:30,Premier League,Matchweek 35,Sat,Home,D,2,2,Liverpool,...,82.4,37.8,5253.0,2093.0,280.0,128.0,112.0,34.0,2018,West Bromwich Albion
4717,2018-04-28,15:00,Premier League,Matchweek 36,Sat,Away,W,1,0,Newcastle Utd,...,70.2,51.1,5434.0,2810.0,258.0,115.0,92.0,46.0,2018,West Bromwich Albion
4718,2018-05-05,15:00,Premier League,Matchweek 37,Sat,Home,W,1,0,Tottenham,...,66.7,42.1,3005.0,1837.0,136.0,52.0,48.0,32.0,2018,West Bromwich Albion


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4720 entries, 0 to 4719
Data columns (total 46 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          4720 non-null   object 
 1   Time_x        4720 non-null   object 
 2   Comp          4720 non-null   object 
 3   Round         4720 non-null   object 
 4   Day           4720 non-null   object 
 5   Venue         4720 non-null   object 
 6   Result        4720 non-null   object 
 7   GF            4720 non-null   object 
 8   GA            4720 non-null   object 
 9   Opponent      4720 non-null   object 
 10  xG            4720 non-null   float64
 11  xGA           4720 non-null   float64
 12  Poss          4720 non-null   float64
 13  Attendance    3836 non-null   float64
 14  Captain       4720 non-null   object 
 15  Formation     4720 non-null   object 
 16  Referee       4720 non-null   object 
 17  Match Report  4720 non-null   object 
 18  Notes         0 non-null    

In [5]:
data.to_csv("4720_EPL.csv")